# Multiverse Parser

Make sure the [Multiverse Framework](https://github.com/Multiverse-Framework/Multiverse) is installed properly.

In [1]:
import ipywidgets
from IPython.display import display
import zipfile
import os
import numpy

from multiverse_parser import InertiaSource
from multiverse_parser import UrdfImporter, MjcfImporter, UsdImporter
from multiverse_parser import UrdfExporter, MjcfExporter

uploader = ipywidgets.FileUpload(accept='.zip', multiple=True)
label = ipywidgets.Label("")
usda_widget = ipywidgets.Checkbox(value=True, description='USDA', disabled=False, indent=False)
urdf_widget = ipywidgets.Checkbox(value=False, description='URDF', disabled=False, indent=False)
mjcf_widget = ipywidgets.Checkbox(value=False, description='MJCF', disabled=False, indent=False)
hbox_widget = ipywidgets.HBox([ipywidgets.Label("Export as:"), ipywidgets.VBox([usda_widget, urdf_widget, mjcf_widget])])
visual_options = ipywidgets.RadioButtons( options=['with_visual','no_visual'], value='with_visual', description='Visual:', disabled=False)
collision_options = ipywidgets.RadioButtons( options=['with_collision','no_collision'], value='with_collision', description='Collision:', disabled=False)
physics_options = ipywidgets.RadioButtons( options=['with_physics','no_physics'], value='with_physics', description='Physics:', disabled=False)
fixed_base_options = ipywidgets.Checkbox(value=True, description='Fix based', disabled=False, indent=False)
inertiasource_options = ipywidgets.ToggleButtons(options=['From source', 'From visual meshes', 'From collision meshes'], description='Inertia source:', disabled=False, button_style='')
add_xform_for_each_geom_options = ipywidgets.Checkbox(value=False, description='Add body for each geom (only for input USD)', disabled=False, indent=False)
default_color_option = ipywidgets.ColorPicker(concise=False, description='Color', value='red', disabled=False)
default_collision_opacity_option = ipywidgets.FloatSlider(
    value=0.0,
    min=0,
    max=1.0,
    step=0.1,
    description='Opacity:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)
hbox_options_widget = ipywidgets.VBox([ipywidgets.HBox([visual_options, collision_options, physics_options]), 
                                       ipywidgets.VBox([fixed_base_options, add_xform_for_each_geom_options, inertiasource_options]),
                                       ipywidgets.Label("Default mesh RGBA:"),
                                       ipywidgets.HBox([default_color_option, default_collision_opacity_option])])

labels = [label]
labels_widget = ipywidgets.HBox(labels)
output_dir = os.path.join(os.getcwd(), "..", "output")

def hex_to_rgb(hex_color):
    # Remove the '#' character if present
    hex_color = hex_color.lstrip('#')
    
    # Convert hex to RGB tuple
    rgb_color = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

    # Normalize the RGB values to the range 0-1
    rgb_normalized = tuple(c / 255.0 for c in rgb_color)
    
    return rgb_normalized

def convert(file):
    in_extension = os.path.splitext(file)[1]
    fixed_base = fixed_base_options.value
    with_physics = physics_options.value == "with_physics"
    with_visual = visual_options.value == "with_visual"
    with_collision = collision_options.value == "with_collision"
    rgb = default_color_option.value
    if inertiasource_options.value == "From visual meshes":
        inertia_source = InertiaSource.FROM_VISUAL_MESH
    elif inertiasource_options.value == "From collision meshes":
        inertia_source = InertiaSource.FROM_COLLISION_MESH
    elif inertiasource_options.value == "From source":
        inertia_source = InertiaSource.FROM_SRC
    
    if rgb == "red":
        r = 1.0
        g = 0.0
        b = 0.0
    else:
        r, g, b = hex_to_rgb(rgb)
        
    a = float(default_collision_opacity_option.value)
    default_rgba = numpy.array([r, g, b, a])
    
    if in_extension in [".usd", ".usda", ".usdc"]:
        factory = UsdImporter(file_path=file,
                              fixed_base=fixed_base,
                              with_physics=with_physics,
                              with_visual=with_visual,
                              with_collision=with_collision,
                              inertia_source=inertia_source,
                              default_rgba=default_rgba,
                              add_xform_for_each_geom=add_xform_for_each_geom.value)
    elif in_extension == ".urdf":
        factory = UrdfImporter(file_path=file,
                               fixed_base=fixed_base,
                               with_physics=with_physics,
                               with_visual=with_visual,
                               with_collision=with_collision,
                               inertia_source=inertia_source,
                               default_rgba=default_rgba)
    elif in_extension == ".xml":
        factory = MjcfImporter(file_path=file,
                               fixed_base=fixed_base,
                               with_physics=with_physics,
                               with_visual=with_visual,
                               with_collision=with_collision,
                               inertia_source=inertia_source,
                               default_rgba=default_rgba)
    else:
        raise NotImplementedError(f"Importing from {in_extension} is not supported yet.")

    factory.import_model()

    output_file_name = os.path.splitext(os.path.basename(file))[0]

    if usda_widget.value:
        output_usd_path = os.path.join(output_dir, output_file_name, output_file_name + ".usda")
        factory.save_tmp_model(usd_file_path=str(output_usd_path))
    if mjcf_widget.value:
        exporter = MjcfExporter(file_path=os.path.join(output_dir, output_file_name, output_file_name + ".xml"),
                                factory=factory)
        exporter.build()
        exporter.export(keep_usd=False)
    if urdf_widget.value:
        exporter = UrdfExporter(file_path=os.path.join(output_dir, output_file_name, output_file_name + ".urdf"),
                                factory=factory)
        exporter.build()
        exporter.export(keep_usd=False)

def find_first_file_with_extension(root_dir, extensions):
    # Walk through the directory and look for the first file with the given extensions
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(extensions):
                return os.path.join(root, file)
    return None

def on_upload_change(change):
    if len(uploader.value) != 1:
        label.value = "Please upload exactly one file!"
        return

    uploaded_file = uploader.value[0]
    
    # Extract file content and metadata
    file_name = uploaded_file['name']
    file_path = os.path.join("/tmp", file_name)    
    file_content = uploaded_file['content']

    # Save the zip file to disk
    with open(file_path, 'wb') as f:
        f.write(file_content)

    # Unzip the file
    try:
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            # Create a directory to extract files into
            extract_dir = os.path.join("/tmp", file_name.replace(".zip", ""))
            os.makedirs(extract_dir, exist_ok=True)
            
            # Extract all the contents
            zip_ref.extractall(extract_dir)
            
            label.value = f"File '{file_name}' uploaded and extracted to {extract_dir} successfully!"

            # Search for the first file with the desired extension
            first_file = find_first_file_with_extension(extract_dir, ('.urdf', '.xml', '.usda'))
            
            if first_file:
                label.value += f" Found {os.path.basename(first_file)} to be converted"
                convert(first_file)
            else:
                label.value = "No .urdf, .xml, or .usda files found."
    except zipfile.BadZipFile:
        label.value = "The uploaded file is not a valid zip file."    

uploader.observe(on_upload_change, names='value')

display(uploader, hbox_widget, hbox_options_widget, label)

FileUpload(value=(), accept='.zip', description='Upload', multiple=True)

Label(value='')